In [1]:
import tensorflow as tf
from datetime import datetime
%load_ext tensorboard


In [44]:
from tensorboard import main as tb
tf.flags.FLAGS.logdir = "/path/to/graphs/"
tb.main()

AttributeError: module 'tensorflow' has no attribute 'flags'

In [2]:
# Models, layers and modules
# learning how to define, save, restore a model

# What is a model?
# function that computes something in tensors (a forward pass)
# Some variable that can be updated

In [3]:
# Defining models and layers in tensorflow
# in tensorflow, most high-level implementations are built on tf.Module class

class SimpleModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)
    self.a_variable = tf.Variable(5.0, name="train_me")
    self.non_trainable_variable = tf.Variable(5.0, trainable=False, name="do_not_train_me")
  def __call__(self, x):
    return self.a_variable * x + self.non_trainable_variable

simple_module = SimpleModule(name="simple")

simple_module(tf.constant(5.0))

print("Trainable variables: " , simple_module.trainable_variables)
print("all variables: ", simple_module.variables)

Trainable variables:  (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>,)
all variables:  (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>, <tf.Variable 'do_not_train_me:0' shape=() dtype=float32, numpy=5.0>)


In [4]:
# two-layer linear layer model made out of modules

# model with two dense layers


# creates random weight and zero biases when initialized when initialized.
# weights and biases are trainable variables.

class Dense(tf.Module):
    def __init__(self, in_features, out_features, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(
            tf.random.normal([in_features, out_features]), name='w'
        )
        self.b = tf.Variable(tf.zeros([out_features], name='b'))
    def __call__ (self, x):
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)

# Module containing two dense layers.
# when called, this module will pass two variables through two dense layers
# and return a result.

class SequentialModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)

        self.dense_1 = Dense(in_features=3, out_features=3)
        self.dense_2 = Dense(in_features=3, out_features=2)

    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

my_model = SequentialModule(name = "the_model")

print("Model result : ", my_model(tf.constant([[2.0, 2.0, 2.0]])))

print("Submodules : ", my_model.submodules)


for var in my_model.variables:
    print(var, "\n")


Model result :  tf.Tensor([[2.8003173 0.       ]], shape=(1, 2), dtype=float32)
Submodules :  (<__main__.Dense object at 0x000002309F9D8EC8>, <__main__.Dense object at 0x00000230D9AE5E48>)
<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 3) dtype=float32, numpy=
array([[-1.3608024 , -0.73394036, -0.6596383 ],
       [ 0.4100763 , -2.9095657 ,  0.52398837],
       [-1.3964335 , -0.22095565,  1.0533845 ]], dtype=float32)> 

<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
array([[-0.01071412,  1.1030895 ],
       [-0.5779114 , -1.2406939 ],
       [ 1.5256683 , -0.65782696]], dtype=float32)> 



In [5]:
# Waiting to create variables
# with deferring variable creation to first moduel call,
# we don't have to pre-specify variable dimension.

# this is why we need to specify only the output dimesion in keras.Dense layers

class FlexibleDenseModule(tf.Module):
    def __init__(self, out_features, name=None):
        super().__init__(name=name)
        self.is_built = False
        self.out_features = out_features

    def __call__(self, x):
        if not self.is_built:
            self.w = tf.Variable(
                tf.random.normal([x.shape[-1], self.out_features]), name='w'
            )
            self.b = tf.Variable(tf.zeros([self.out_features]), name = 'b')
            self.is_built = True
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)

class MySequentialModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)

        self.dense_1 = FlexibleDenseModule(out_features=3)
        self.dense_2 = FlexibleDenseModule(out_features=2)

    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)


my_model = MySequentialModule(name = "the_model")
print("Module result :", my_model(tf.constant([[2.0,2.0,2.0]])))

Module result : tf.Tensor([[0. 0.]], shape=(1, 2), dtype=float32)


In [6]:
# saving weights with Checkpoints / SavedModel

# Checkpoints are only weights in the model
# this will create file in current directory

chkp_path = "my_checkpoint"
checkpoint = tf.train.Checkpoint(model = my_model)
checkpoint.write(chkp_path)

# in this directory,
# my_checkpoint.data-00000-of-000000 and
# my_checkpoint.index
# files are created.

# to look inside a checkpoint
tf.train.list_variables(chkp_path)

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('model/dense_1/b/.ATTRIBUTES/VARIABLE_VALUE', [3]),
 ('model/dense_1/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 3]),
 ('model/dense_2/b/.ATTRIBUTES/VARIABLE_VALUE', [2]),
 ('model/dense_2/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 2])]

In [7]:
# you can load model with checkpoint

new_model = MySequentialModule()
new_checkpoint = tf.train.Checkpoint(model=new_model)
new_checkpoint.restore("my_checkpoint")

new_model(tf.constant([[2.0,2.0,2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0., 0.]], dtype=float32)>

In [47]:
# Saving models

# Two main APIs.
# one for high level(keras) : model.save / tf.keras.models.load_model
# another for low level(tf) : tf.saved_model.save / tf.saved_model.load

# Running models without original python objects (e.g MysequentialModule)
# using GRAPHS

# this graph contains operations(ops)
class MySequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = Dense(in_features=3, out_features=3)
    self.dense_2 = Dense(in_features=3, out_features=2)

  @tf.function
  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# You have made a model with a graph!
my_model = MySequentialModule(name="the_model")


NameError: name 'tensorflow' is not defined

In [43]:
# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/func/%s" % stamp
writer = tf.summary.create_file_writer(logdir)

# Create a new model to get a fresh trace
# Otherwise the summary will not see the graph.
new_model = MySequentialModule()

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True)
tf.profiler.experimental.start(logdir)
# Call only one tf.function when tracing.
z = print(new_model(tf.constant([[2.0, 2.0, 2.0]])))
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

tf.Tensor([[3.7276113 3.152882 ]], shape=(1, 2), dtype=float32)


In [41]:
print(my_model([[2.0, 2.0, 2.0]]))

tf.Tensor([[0.        3.4496217]], shape=(1, 2), dtype=float32)


In [ ]:
tf.saved_model.save(my_model, "the_saved_model")

In [67]:
# loading the saved model

loaded_model = tf.saved_model.load("the_saved_model")

# loaded_model = tf.keras.models.load_model("the_graph_model")
# print(loaded_model([[2.0,2.0,2.0]]))
loaded_model(tf.constant([[2.0,2.0,2.0]]))

ValueError: Found zero restored functions for caller function.

In [48]:
# Keras layers

class MyDense(tf.keras.layers.Layer):
    def __init__(self, in_features, out_features, **kwargs):
        super().__init__(**kwargs)

        self.w = tf.Variable(tf.random.normal([in_features, out_features]), name='w')
        self.b = tf.Variable(tf.zeros([out_features]), name='b')

    def call(self, x):
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)

simple_layer = MyDense(name='simple', in_features=3, out_features=3)

simple_layer([[2.0, 2.0, 2.0]])

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[1.1913686, 0.       , 0.       ]], dtype=float32)>

In [49]:
# to gain flexibility like above, we can use 'build' in keras.

class FlexibleDense(tf.keras.layers.Layer):
    def __init__(self, out_features, **kwargs):
        super().__init__(**kwargs)
        self.out_features = out_features

    def build(self, input_shape):
        self.w = tf.Variable(tf.random.normal([input_shape[-1], self.out_features]), name='w')
        self.b = tf.Variable(tf.zeros([self.out_features]), name='b')

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


flexible_dense = FlexibleDense(out_features=3)

flexible_dense.variables

[]

In [50]:
# Call it, with predictably random results
print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0], [3.0, 3.0, 3.0]])))

Model results: tf.Tensor(
[[3.209397  3.7270632 0.8851066]
 [4.8140955 5.5905952 1.3276601]], shape=(2, 3), dtype=float32)


In [51]:
flexible_dense.variables

[<tf.Variable 'flexible_dense/w:0' shape=(3, 3) dtype=float32, numpy=
 array([[0.19169681, 0.37988058, 0.01263723],
        [0.08726376, 1.0585585 , 0.25036457],
        [1.325738  , 0.42509264, 0.17955151]], dtype=float32)>,
 <tf.Variable 'flexible_dense/b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

In [52]:
# After being built once, input shape different with built variables will be rejected.
try:
  print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0, 2.0]])))
except tf.errors.InvalidArgumentError as e:
  print("Failed:", e)


Failed: Exception encountered when calling layer "flexible_dense" (type FlexibleDense).

Matrix size-incompatible: In[0]: [1,4], In[1]: [3,3] [Op:MatMul]

Call arguments received:
  • inputs=tf.Tensor(shape=(1, 4), dtype=float32)


In [53]:
class MySequentialModel(tf.keras.Model):
    def __init__(self, name=None, **kwargs):
        super().__init__(**kwargs)

        self.dense_1 = FlexibleDense(out_features=3)
        self.dense_2 = FlexibleDense(out_features=2)
    def call(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

my_sequential_model = MySequentialModel(name="the model")

print("Model results:", my_sequential_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([[-0.72177374  1.5122838 ]], shape=(1, 2), dtype=float32)


In [55]:
my_sequential_model.variables

[<tf.Variable 'my_sequential_model/flexible_dense_1/w:0' shape=(3, 3) dtype=float32, numpy=
 array([[ 0.46336198,  0.34460852, -0.67476064],
        [-0.3470996 , -1.7257942 , -0.543754  ],
        [-1.135253  , -1.6934762 ,  0.8563872 ]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_1/b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_2/w:0' shape=(3, 2) dtype=float32, numpy=
 array([[-0.71941096, -0.6554676 ],
        [ 0.33602536,  0.09209906],
        [ 0.16788429, -1.0256059 ]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_2/b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

In [56]:
my_sequential_model.submodules

(<__main__.FlexibleDense at 0x231990f8cc8>,
 <__main__.FlexibleDense at 0x231997a5948>)

In [59]:
my_sequential_model.summary()

Model: "my_sequential_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flexible_dense_1 (FlexibleD  multiple                 12        
 ense)                                                           
                                                                 
 flexible_dense_2 (FlexibleD  multiple                 8         
 ense)                                                           
                                                                 
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


In [58]:
# with functional API

inputs = tf.keras.Input(shape = [3,])

x = FlexibleDense(3)(inputs)
x = FlexibleDense(2)(x)

my_funtional_model = tf.keras.Model(inputs=inputs, outputs=x)

my_funtional_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3)]               0         
                                                                 
 flexible_dense_5 (FlexibleD  (None, 3)                12        
 ense)                                                           
                                                                 
 flexible_dense_6 (FlexibleD  (None, 2)                8         
 ense)                                                           
                                                                 
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


In [62]:
# saving keras models
my_sequential_model.save("keras_saved_model")
reconstructed_model = tf.keras.models.load_model("keras_saved_model/")

INFO:tensorflow:Assets written to: keras_saved_model\assets


INFO:tensorflow:Assets written to: keras_saved_model\assets


In [63]:
reconstructed_model(tf.constant([[2.0,2.0,2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-0.72177374,  1.5122838 ]], dtype=float32)>

In [65]:
# testing with tf.saved_model.save

tf.saved_model.save(my_sequential_model, "tf_saved_model")
reconstructed_tf_model = tf.saved_model.load("tf_saved_model/")

INFO:tensorflow:Assets written to: tf_saved_model\assets


INFO:tensorflow:Assets written to: tf_saved_model\assets


In [66]:
reconstructed_tf_model(tf.constant([[2.0,2.0,2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-0.72177374,  1.5122838 ]], dtype=float32)>